In [1]:
! pip install azure-search-documents --pre
! pip install openai
! pip install python-dotenv

In [2]:
# Import required libraries  
import os  
import json  
import openai  
from dotenv import load_dotenv  
from tenacity import retry, wait_random_exponential, stop_after_attempt  
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient  
from azure.search.documents.indexes import SearchIndexClient  
from azure.search.documents.models import Vector  
from azure.search.documents.indexes.models import (  
    SearchIndex,  
    SearchField,  
    SearchFieldDataType,  
    SimpleField,  
    SearchableField,  
    SearchIndex,  
    SemanticConfiguration,  
    PrioritizedFields,  
    SemanticField,  
    SearchField,  
    SemanticSettings,  
    VectorSearch,  
    HnswVectorSearchAlgorithmConfiguration, 
)  

from langchain.document_loaders import AzureBlobStorageContainerLoader
#from dotenv import load_dotenv

from langchain.vectorstores.azuresearch import AzureSearch
from langchain.document_loaders import AzureBlobStorageContainerLoader
from langchain.text_splitter import CharacterTextSplitter
from azure.storage.blob import BlobServiceClient
import tempfile
from langchain.document_loaders import PyPDFLoader

from langchain.embeddings.openai import OpenAIEmbeddings
  
# Configure environment variables  
load_dotenv()  
service_endpoint = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT") 
index_name = os.getenv("AZURE_SEARCH_INDEX_NAME") 
key = os.getenv("AZURE_SEARCH_ADMIN_KEY") 
openai.api_type = "azure"  
openai.api_key = os.getenv("AZURE_OPENAI_API_KEY")  
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")  
openai.api_version = os.getenv("AZURE_OPENAI_API_VERSION")  
credential = AzureKeyCredential("jA50VnyT0OAeuHpY0RxJGz7z9kfrmdj7YF5PHYiPXjAzSeBS6hJg")

In [3]:
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_BASE"] = "https://testingchat.openai.azure.com/"
os.environ["OPENAI_API_KEY"] = "e8143eabb02541259054426630db12a7"
os.environ["OPENAI_API_VERSION"] = "2023-05-15"
model: str = "text-embedding-ada-002"
pdf_file_name = "Alaska Airlines and Horizon Air SMS Manual.pdf"   
#e8143eabb02541259054426630db12a7    

#vector_store_address: str = "https://genaisafety.search.windows.net"
#vector_store_password: str = "iDXpfwNsnwyN88dYUlj3c0TaXP76z1DlWlSxZ9QmPxAzSeAu7ppU"

AZURE_SEARCH_SERVICE_ENDPOINT = "https://genaisafety.search.windows.net"
AZURE_SEARCH_ADMIN_KEY = "jA50VnyT0OAeuHpY0RxJGz7z9kfrmdj7YF5PHYiPXjAzSeBS6hJg"

#Creating Serach index

AZURE_SEARCH_INDEX_NAME = str(pdf_file_name.split(".")[0].lower().replace(" ","-") + "-index")

In [4]:
#pip install --upgrade azure-search-documents
embeddings = OpenAIEmbeddings(deployment = model,chunk_size=1) 
vector_store: AzureSearch = AzureSearch(
    azure_search_endpoint=AZURE_SEARCH_SERVICE_ENDPOINT,
    azure_search_key=AZURE_SEARCH_ADMIN_KEY,
    index_name = AZURE_SEARCH_INDEX_NAME,
    embedding_function=embeddings.embed_query,
)

In [5]:
AZURE_SEARCH_INDEX_NAME

'alaska-airlines-and-horizon-air-sms-manual-index'

In [6]:
def pdf_text_extraction(pdf_file_path):
    if pdf_file_path is not None:
        # Open the PDF file and read its contents
        with open(pdf_file_path, 'rb') as pdf_file:
            # Create a temporary file to write the PDF contents
            with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
                tmp_file.write(pdf_file.read())
                pdf_path = tmp_file.name

        # Now you can use the PyPDFLoader on the temporary PDF file
        loader = PyPDFLoader(pdf_path)
        pages_split = loader.load_and_split()
    return pages_split

In [ ]:
from azure.identity import DefaultAzureCredential
#from azure.search.documents.indexes import SearchServiceClient
from azure.search.documents import SearchServiceClient
#from azure.search.documents.indexes.models import SearchFieldDataType, VectorSearch, HnswVectorSearchAlgorithmConfiguration

# Azure Cognitive Search service endpoint and admin API key
service_endpoint = AZURE_SEARCH_SERVICE_ENDPOINT
admin_key = AZURE_SEARCH_ADMIN_KEY

# Create a SearchServiceClient
credential = DefaultAzureCredential()
search_service_client = SearchServiceClient(endpoint=service_endpoint, credential=credential)

# Define your vector search algorithm configuration
vector_search_config = HnswVectorSearchAlgorithmConfiguration(
    name="my-vector-config",
    kind="hnsw",
    parameters={
        "m": 32,  # Customize these parameters as needed
        "efConstruction": 200,
        "efSearch": 300,
        "metric": "cosine"
    }
)





# Create the vector search algorithm configuration
search_service_client.create_or_update_indexer(skillsetName="my-vector-config", indexerName="my-vector-config", skillset=vector_search_config)

print("Vector search configuration 'my-vector-config' created successfully.")


In [27]:
# Replace with your actual connection string
connection_string = "DefaultEndpointsProtocol=https;AccountName=storageaccountgenai;AccountKey=WAfyL4uqWgarFdx1ibNgWv9lmOINODLuN6nnLSQLgE/iuHhKGi1pYd6NQJ6LBnZO/DnnQfhbNSWi+AStsOLf1Q==;EndpointSuffix=core.windows.net"

# Replace with your container name
container_name = "safety"


# Create a BlobServiceClient using the connection string
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

# Get a reference to the container
container_client = blob_service_client.get_container_client(container_name)

embeddings = OpenAIEmbeddings(deployment = model,chunk_size=1)

# List blobs in the container
blobs = container_client.list_blobs()
dict_data = []
for blob in blobs:
    #print(blob)
    if blob.name.startswith("manuals") and blob.name.endswith(".pdf"):  # Check if the blob is a PDF file
        blob_client = container_client.get_blob_client(blob)
        print(blob.name)
        try:
            pdf_files = blob_client.download_blob().readall()
                # Save the blob content to a local PDF file or process it as needed
            pdf_name = blob.name.split('/')[1]
            print(pdf_name)
            with open(pdf_name, "wb") as pdf_file:
                pdf_file.write(pdf_files)
                print("PDF file downloaded successfully.")
        except Exception as e:
             print(f"An error occurred: {e}")       
            
        
        #pdf_name=str(blob.name).split(".")[0]
        #pdf_name = "AAG_Safety_Manual.pdf"
        pdf_file_name=str(blob.name).split("/")[1]
        #loader = PyPDFLoader(pdf_name)
        loader = pdf_text_extraction(pdf_file_name)
        
        documents = loader #.load()
        #print(documents)
        text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
        docs = text_splitter.split_documents(documents)
        print("PDF_NAME, length", blob.name, len(docs))
        #for index, doc in enumerate(docs):
        #    dict_data[index] = {}  # Create a sub-dictionary for each document
        #    dict_data[index]['id'] = blob.name
        #    dict_data[index]['page_content'] = doc.page_content
        #   dict_data[index]['page_number'] = doc.metadata['page']
        
        bid=str(blob.name).split("/")[1].split(".")[0].lower().replace(" ","_").replace("_","-")
            
        for doc in docs:
            dict_data.append({
                'id': bid,
                'page_content': doc.page_content,  # You can set a title based on your needs
                'page_number': doc.metadata['page'],
            })

        print(f"splitted into {len(docs)} documents") 
        
        

        for item in dict_data:
            title = item['id']
            content = item['page_content']
            title_embeddings = embeddings.embed_query(title)
            content_embeddings = embeddings.embed_query(content)
            item['titleVector'] = title_embeddings
            item['contentVector'] = content_embeddings
            
            

        vector_search = VectorSearch(
            algorithm_configurations=[
                HnswVectorSearchAlgorithmConfiguration(
                    name="my-vector-config",
                    kind="hnsw",
                    parameters={
                        "m": 4,
                        "efConstruction": 400,
                        "efSearch": 500,
                        "metric": "cosine"
                    }
                )
            ]
        )
        
        
        semantic_config = SemanticConfiguration(
            name="my-semantic-config",
            prioritized_fields=PrioritizedFields(
                title_field=SemanticField(field_name="id"),
                prioritized_content_fields=[SemanticField(field_name="page_content")]
            )
        )

        # Create the semantic settings with the configuration
        semantic_settings = SemanticSettings(configurations=[semantic_config])


        
        #credential = AzureKeyCredential(AZURE_SEARCH_ADMIN_KEY)
        search_client = SearchIndexClient(endpoint=AZURE_SEARCH_SERVICE_ENDPOINT, credential=credential)
        search_client_v2 = SearchClient(endpoint=AZURE_SEARCH_SERVICE_ENDPOINT, index_name=AZURE_SEARCH_INDEX_NAME, credential=credential)

        bid=str(blob.name).split("/")[1].split(".")[0].lower().replace(" ","_").replace("_","-")
        AZURE_SEARCH_INDEX_NAME=bid + "-index"
        print(AZURE_SEARCH_INDEX_NAME)
        # Define your Azure Cognitive Search credentials and index settings as previously done
        # Check if the index already exists
        existing_index = None
        # Define your Azure Cognitive Search credentials and index settings as previously done


            # The index doesn't exist, so create a new one
        if existing_index == None:
            #index = SearchIndex(name=AZURE_SEARCH_INDEX_NAME,fields=[SimpleField(name="id", type=SearchFieldDataType.String, key=True, searchable=False),SearchableField(name="content", type=SearchFieldDataType.String, searchable=True),SearchableField(name="content_vector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single), searchable=True, vector_search_dimensions=1536,vector_search_configuration="my-vector-config",),SearchableField(name="metadata", type=SearchFieldDataType.String, searchable=True),],)
            #index = SearchIndex(name=AZURE_SEARCH_INDEX_NAME,fields=[SimpleField(name="id", type=SearchFieldDataType.String, key=True, searchable=False),SearchableField(name="content", type=SearchFieldDataType.String, searchable=True),SearchableField(name="metadata", type=SearchFieldDataType.String, searchable=True),],)
            fields = [
            SimpleField(name="id", type=SearchFieldDataType.String, key=True, sortable=True, filterable=True, facetable=True),
            SearchableField(name="page_content", type=SearchFieldDataType.String),
            SearchableField(name="page_number", type=SearchFieldDataType.String),
            SearchField(name="titleVector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                        searchable=True, vector_search_dimensions=1536, vector_search_configuration="my-vector-config"),
            SearchField(name="contentVector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                        searchable=True, vector_search_dimensions=1536, vector_search_configuration="my-vector-config"),
            ]

            index = SearchIndex(name=AZURE_SEARCH_INDEX_NAME, fields=fields,
                    vector_search=vector_search)

            # Create the new index
            search_client.create_index(index)
            #print(f"Index '{AZURE_SEARCH_INDEX_NAME}' created successfully.")
            #result = search_client_v2.upload_documents(documents=final_doc)
            print("Vector store added successfully")

        else:
            try:
                # Check if the index exists
                existing_index = search_client.get_index(AZURE_SEARCH_INDEX_NAME)
                print(f"Index '{AZURE_SEARCH_INDEX_NAME}' already exists. You can use the existing index.")
                #vector_store.add_documents(documents=final_doc)
                result = search_client.upload_documents(documents=final_doc)
                print(f"Uploaded {len(content)} documents")
                print("Vector store added successfully")
            except Exception as e:
                print(e)
                pass
    else:
        continue

manuals/AAG_Safety_Manual.pdf
AAG_Safety_Manual.pdf
PDF file downloaded successfully.
PDF_NAME, length manuals/AAG_Safety_Manual.pdf 332
splitted into 332 documents
aag-safety-manual-index
Vector store added successfully
manuals/Alaska Airlines and Horizon Air SMS Manual.pdf
Alaska Airlines and Horizon Air SMS Manual.pdf
PDF file downloaded successfully.
PDF_NAME, length manuals/Alaska Airlines and Horizon Air SMS Manual.pdf 125
splitted into 125 documents
alaska-airlines-and-horizon-air-sms-manual-index
Vector store added successfully


In [28]:
vector_search = VectorSearch(
    algorithm_configurations=[
        HnswVectorSearchAlgorithmConfiguration(
            name="my-vector-config",
            kind="hnsw",
            parameters={
                "m": 4,
                "efConstruction": 400,
                "efSearch": 500,
                "metric": "cosine"
            }
        )
    ]
)


semantic_config = SemanticConfiguration(
    name="my-semantic-config",
    prioritized_fields=PrioritizedFields(
        title_field=SemanticField(field_name="id"),
        prioritized_keywords_fields=[SemanticField(field_name="page_content")]
    )
)

# Create the semantic settings with the configuration
semantic_settings = SemanticSettings(configurations=[semantic_config])

# Create the search index with the semantic settings
index = SearchIndex(name=AZURE_SEARCH_INDEX_NAME, fields=fields,
                    vector_search=vector_search, semantic_settings=semantic_settings)

result = search_client.create_or_update_index(index)
print(f' {result.name} created index')

 alaska-airlines-and-horizon-air-sms-manual-index created


In [29]:
print(type(str(dict_data[0]['page_number'])))

<class 'str'>


In [73]:
final_doc=[]
for index,doc in enumerate(docs):
    final_doc.append({'id':dict_data[index]['id'],'page_content':dict_data[index]['page_content'],'page_number':str(dict_data[index]['page_number']), 'titleVector':dict_data[index]['titleVector'], 'contentVector':dict_data[index]['contentVector']})
    search_client_azure.upload_documents(final_doc)

In [71]:
search_client_azure = SearchClient(endpoint=AZURE_SEARCH_SERVICE_ENDPOINT, index_name=AZURE_SEARCH_INDEX_NAME, credential=credential)
for doc in final_doc:
    search_client_azure.upload_documents(doc)  
print(f"Uploaded {len(final_doc)} documents") 

Uploaded 125 documents


In [54]:
result=search_client_azure.upload_documents(final_doc)  

In [61]:
print(dict_data[0])

{'id': 'aag-safety-manual', 'page_content': 'Uncontrolled copy when downloaded or printed from an electronic document.\nSafety Manual\n(Alaska Air Group)\nRevision 1-2023 6/16/2023', 'page_number': 0, 'titleVector': [-0.012831310985940949, -0.011140199590267756, -0.004548386291599466, -0.03244116219068729, 0.002737487252775625, 0.016389692514307743, -0.01277494082339378, 0.00816666071021899, -0.027128251786043823, -0.0014295180134742038, 0.010449662072266521, 0.008526022708348345, -0.0030669018744560205, 0.0284106792704194, 0.00638394721833003, -0.00873036512965845, 0.020955695238402804, 0.00663056749438116, -0.010442615685532801, -0.014000997213899595, -0.007056868769496965, 0.022830009895184942, -0.020941602464935365, -0.023337344152077232, 0.00503105772926244, 0.008547160937226912, 0.007440892189871747, -0.035372421819011145, -0.004523724403692741, -0.010921764395490212, -0.00035077360311718144, -0.024760695645836838, -0.0009168997232312293, -0.022731362343558045, -0.010787883978872

In [63]:
def setup_vector_store(AZURE_SEARCH_SERVICE_ENDPOINT, AZURE_SEARCH_ADMIN_KEY, AZURE_SEARCH_INDEX_NAME,embeddings):    
    return AzureSearch(
        azure_search_endpoint=AZURE_SEARCH_SERVICE_ENDPOINT,
        azure_search_key=AZURE_SEARCH_ADMIN_KEY,
        index_name=AZURE_SEARCH_INDEX_NAME,
        embedding_function=embeddings.embed_query,
    )

In [69]:
vector_store.add_documents(documents=final_doc)

AttributeError: 'dict' object has no attribute 'page_content'

In [64]:
aag_vector_store = setup_vector_store(AZURE_SEARCH_SERVICE_ENDPOINT, AZURE_SEARCH_ADMIN_KEY,
                                      AZURE_SEARCH_INDEX_NAME, embeddings,)

In [62]:
vector_store.add_documents(documents=dict_data)

AttributeError: 'dict' object has no attribute 'page_content'

In [ ]:
# Pure Vector Search
query = "software"  


# Create a Vector object with the embeddings result
vector = Vector(value=embeddings_result, k=3, fields="contentVector")

results = search_client_v2.search(  
    search_text=None,  
    vectors=[vector],
    select=["id", "page_content", "page_number"],
)  

for result in results:  
    print(f"Title: {result['id']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['page_content']}")  
    print(f"Category: {result['page_number']}\n")


In [40]:
# Pure Vector Search
query = "software"  
  
#search_client = SearchClient(service_endpoint, index_name, credential=credential)
vector = Vector(value=content_embeddings, k=3, fields="contentVector")
  
results = search_client_v2.search(  
    search_text=None,  
    vectors= [vector],
    select=["id", "page_content", "page_number"],
)  
  
for result in results:  
    print(f"Title: {result['id']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['page_content']}")  
    print(f"Category: {result['page_number']}\n")  

In [41]:
results

<iterator object azure.core.paging.ItemPaged at 0x133189910>

In [ ]:
dict_data

In [ ]:
# Convert the dictionary to a JSON string
input_data = json.dumps(dict_data)

# json_string contains the JSON representation of the dictionary
#print(input_data)

In [ ]:
embeddings = OpenAIEmbeddings(deployment = model,chunk_size=1)
#@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
# Function to generate embeddings for title and content fields, also used for query embeddings
def generate_embeddings(text):
    response = openai.Embedding.create(
        input=text, engine="text-embedding-ada-002")
    embeddings = response['data'][0]['embedding']
    return embeddings

# Generate embeddings for title and content fields
for item in dict_data:
    title = item['id']
    content = item['page_content']
    title_embeddings = embeddings.embed_query(title)
    content_embeddings = embeddings.embed_query(content)
    item['titleVector'] = title_embeddings
    item['contentVector'] = content_embeddings


In [ ]:
# Create a search index
index_client = SearchClient(endpoint=AZURE_SEARCH_SERVICE_ENDPOINT, index_name=AZURE_SEARCH_INDEX_NAME, credential=credential)
search_client = SearchIndexClient(endpoint=AZURE_SEARCH_SERVICE_ENDPOINT, credential=credential)

fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True, sortable=True, filterable=True, facetable=True),
    SearchableField(name="page_content", type=SearchFieldDataType.String),
    SearchField(name="titleVector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                searchable=True, vector_search_dimensions=1536, vector_search_configuration="my-vector-config"),
    SearchField(name="contentVector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                searchable=True, vector_search_dimensions=1536, vector_search_configuration="my-vector-config"),
]

vector_search = VectorSearch(
    algorithm_configurations=[
        HnswVectorSearchAlgorithmConfiguration(
            name="my-vector-config",
            kind="hnsw",
            parameters={
                "m": 4,
                "efConstruction": 400,
                "efSearch": 500,
                "metric": "cosine"
            }
        )
    ]
)

semantic_config = SemanticConfiguration(
    name="my-semantic-config",
    prioritized_fields=PrioritizedFields(
        title_field=SemanticField(field_name="title"),
        prioritized_content_fields=[SemanticField(field_name="content")]
    )
)

# Create the semantic settings with the configuration
semantic_settings = SemanticSettings(configurations=[semantic_config])


# Create the search index with the semantic settings
index = SearchIndex(name=AZURE_SEARCH_INDEX_NAME, fields=fields,
                    vector_search=vector_search, semantic_settings=semantic_settings)
result = search_client.create_or_update_index(index)
print(f' {result.name} created')